In [ ]:
#@title Mount google drive

from google.colab import drive
drive.mount("/content/drive", force_remount=True)

%cd './drive/MyDrive/gaze_estimation'

In [ ]:
import os
import time
import copy
import random
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from tqdm.auto import tqdm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDRegressor, LinearRegression

from util import make_reproducibility, TensorDataset, convert_to_xyz, mae
from networks import *
from menet.variational_em import EM_update
from posthoc import fitrlinear

In [ ]:
def menet_ut_3fold_posthoc(fold, seed, max_iter = 1, path = './menet', experiment_name = 'ut_3fold', device=torch.device('cuda:0')) :
    make_reproducibility(seed + fold)

    train_ids = np.load(f'../UTMultiview_subsampled/ut_dataset/3-fold/fold_{fold}_train_ids.npy').reshape(-1)
    train_images = torch.as_tensor(np.load(f'../UTMultiview_subsampled/ut_dataset/3-fold/fold_{fold}_train_images.npy'), dtype=torch.float)
    train_hps = torch.as_tensor(np.load(f'../UTMultiview_subsampled/ut_dataset/3-fold/fold_{fold}_train_2d_hps.npy'), dtype=torch.float)
    train_gazes = torch.as_tensor(np.load(f'../UTMultiview_subsampled/ut_dataset/3-fold/fold_{fold}_train_2d_gazes.npy'), dtype=torch.float)

    test_ids = np.load(f'../UTMultiview_subsampled/ut_dataset/3-fold/fold_{fold}_test_ids.npy').reshape(-1)
    test_images = torch.as_tensor(np.load(f'../UTMultiview_subsampled/ut_dataset/3-fold/fold_{fold}_test_images.npy'), dtype=torch.float)
    test_hps = torch.as_tensor(np.load(f'../UTMultiview_subsampled/ut_dataset/3-fold/fold_{fold}_test_2d_hps.npy'), dtype=torch.float)
    test_gazes = torch.as_tensor(np.load(f'../UTMultiview_subsampled/ut_dataset/3-fold/fold_{fold}_test_2d_gazes.npy'), dtype=torch.float)

    train_ids_unique = np.unique(train_ids)
    train_m = len(train_ids_unique)
    train_N = len(train_gazes)
    train_cluster = [np.where(train_ids == idx)[0] for idx in train_ids_unique]
    train_n_list = [len(cluster) for cluster in train_cluster]

    test_ids_unique = np.unique(test_ids)
    test_m = len(test_ids_unique)
    test_N = len(test_gazes)
    test_cluster = [np.where(test_ids == idx)[0] for idx in test_ids_unique]


    model = ResNet.ResNet(500,2).to(device)
    model.load_state_dict(torch.load(f'{path}/{experiment_name}_{fold}_selected_model.pt', map_location=device))
    model.eval()

    p=model.p
    K=2

    v_list = [torch.zeros(p, K) for _ in range(train_m)]
    sigma_sq = torch.ones(K)
    Sigma_v = torch.eye(p).repeat(K,1,1)
    train_y_fixed = train_gazes

    with torch.no_grad() :
        # update beta
        # beta = model.fc2.weight.data.clone().T.cpu()

        train_y_list = [train_gazes[cluster] for cluster in train_cluster]
        train_Gamma = torch.zeros(train_N, p)
        for cluster in train_cluster :
            train_Gamma[cluster] = model.get_feature_map(train_images[cluster].to(device), train_hps[cluster].to(device)).detach().cpu()
        train_Gamma_list = [train_Gamma[cluster] for cluster in train_cluster]

        test_Gamma = torch.zeros(test_N, p)
        for cluster in test_cluster :
            test_Gamma[cluster] = model.get_feature_map(test_images[cluster].to(device), test_hps[cluster].to(device)).detach().cpu()

        train_z = train_Gamma[:,1:] # without intercept
        test_z  =  test_Gamma[:,1:] # without intercept

    # EM algorithm
    for iter in range(max_iter) :
        lf1 = SGDRegressor()
        lf2 = SGDRegressor()
        lf1.fit(X = train_z, y = train_y_fixed[:,0])
        lf2.fit(X = train_z, y = train_y_fixed[:,1])

        with torch.no_grad() :
            beta = torch.as_tensor(np.stack([np.concatenate([lf1.intercept_, lf1.coef_]), np.concatenate([lf2.intercept_, lf2.coef_])]), dtype=torch.float).T
            v_list, sigma_sq, Sigma_v = EM_update(train_y_list, train_Gamma_list, beta, sigma_sq, Sigma_v, train_n_list)

            train_fixed  = torch.zeros_like(train_gazes)
            train_random = torch.zeros_like(train_gazes)

            # update fixed parts
            for i in range(train_m) :
                train_fixed[train_cluster[i]] = train_Gamma[train_cluster[i]] @ beta
                train_random[train_cluster[i]] = train_Gamma[train_cluster[i]] @ v_list[i]
            train_y_fixed = train_gazes - train_random


            train_v_list = torch.zeros(K, train_N, p)
            for k in range(K) :
                for i in range(train_m) :
                    train_v_list[k][train_cluster[i]] = v_list[i][:,k].repeat(len(train_cluster[i]), 1)

        w_list = []
        for k in range(K) :
            w_list.append(fitrlinear(X = train_z, y = train_v_list[k], device=device).to(device))
            w_list[k].fit(max_epoch=50)

        with torch.no_grad() :
            test_fixed = test_Gamma @ beta

            test_nu_list = [w_list[k].predict(test_z).detach().cpu() for k in range(K)]

            test_new_adj_1 = torch.zeros_like(test_gazes)
            test_new_adj_1[:,0] = torch.sum(test_Gamma * test_nu_list[0], dim = 1)
            test_new_adj_1[:,1] = torch.sum(test_Gamma * test_nu_list[1], dim = 1)

            test_mean_nu_list = [torch.mean(test_nu, dim=0) for test_nu in test_nu_list]
            test_new_adj_2 = torch.zeros_like(test_gazes)
            test_new_adj_2[:,0] = test_Gamma @ test_mean_nu_list[0]
            test_new_adj_2[:,1] = test_Gamma @ test_mean_nu_list[1]

            w = LinearRegression(fit_intercept=False)
            w.fit(X=train_Gamma.cpu(), y=train_random.cpu())
            w_beta = torch.as_tensor(w.coef_, dtype=torch.float).T


            test_old_adj = test_Gamma @ w_beta

            print(f'Test MAE (fixed) : {mae(test_fixed, test_gazes, is_3d=False, deg=False)}')
            print(f'Test MAE (old) : {mae(test_fixed + test_old_adj, test_gazes, is_3d=False, deg=False)}')
            print(f'Test MAE (new) : {mae(test_fixed + test_new_adj_1, test_gazes, is_3d=False, deg=False)}')
            print(f'Test MAE (new, averaged) : {mae(test_fixed + test_new_adj_2, test_gazes, is_3d=False, deg=False)}')

    return test_Gamma, test_nu_list, test_mean_nu_list, w_list, test_new_adj_1, test_new_adj_2

In [ ]:
seed = 100
path = './menet'
experiment_name = 'ut_3fold'
max_iter=1 # consistent with mpii_loocv
device=torch.device('cuda:0')

posthoc_list = []
for fold in range(3) :
    posthoc_list.append(menet_ut_3fold_posthoc(fold, seed, max_iter, path, experiment_name, device))

In [ ]:
nu_list = [torch.stack(posthoc_list[fold][1]) for fold in range(3)]
mean_nu_list = [torch.stack(posthoc_list[fold][2]) for fold in range(3)]

os.makedirs('./posthoc_results', exist_ok=True)
torch.save(nu_list, f'./posthoc_results/menet_ut_3fold_nu_{max_iter}.pt')
torch.save(mean_nu_list, f'./posthoc_results/menet_ut_3fold_mean_nu_{max_iter}.pt')